In [1]:
import json

HTML = False

def get_dag():
    from pathlib import Path
    from tempfile import TemporaryDirectory

    from daggerml_cli import api, repo
    from tests.util import SimpleApi

    _here = Path(api.__file__).parent.parent
    fn = repo.Resource(str(_here / "tests/fn/sum.py"),
                        adapter='dml-python-in-process-adapter')
    with TemporaryDirectory() as config_dir:
        with SimpleApi.begin('d0', config_dir=config_dir) as d0:
            d0.commit(d0.put_literal(23))
        with SimpleApi.begin('d0.1', config_dir=config_dir) as d0:
            d0.commit(d0.put_literal(24))
        with SimpleApi.begin('d1', config_dir=config_dir) as d1:
            nodes = [
                d1.put_literal(fn, name="fn-resource"),
                d1.put_load('d0', name="node-A"),
                d1.put_load('d0.1', name="node-B", doc="this is the number of times I've been cool"),
                d1.put_literal(1),
                d1.put_literal(23, name="node-D"),
            ]
            tmp = d1.start_fn(*nodes, name="fn_0", doc="the first function call")
            nodes.append(d1.get(tmp, d1.put_literal(1, name="1"), name="get-fn"))
            result = d1.start_fn(*nodes, name="fn_1", doc="the second function call")
            ref = d1.commit(d1.put_literal([*nodes, result], name="final", doc="foo py"))
        if HTML:
            desc = api.write_dag_html(d1.ctx, ref)
        else:
            desc = api.describe_dag(d1.ctx, ref)
    return desc

if HTML:
    with open("./.venv/viz2.html", "w") as f:
        f.write(get_dag())
else:
    print(json.dumps(get_dag(), indent=2))

{
  "id": "0f3f6d31160f185b90db3bd5fd7c3452",
  "expr": null,
  "nodes": [
    {
      "id": "9af5489fba7d8a96ba2de4767aa26351",
      "name": "node-A",
      "doc": null,
      "type": "import",
      "info": "23"
    },
    {
      "id": "15288304534c51f231076559e9baa144",
      "name": null,
      "doc": null,
      "type": "literal",
      "info": "Resource(uri='get', data=None, adapter=None)"
    },
    {
      "id": "f6eb2eeb15b99d7571e9208b69166ce7",
      "name": "fn_0",
      "doc": "the first function call",
      "type": "fn",
      "info": "Resource(uri='/Users/amn/code/daggerml/daggerml-cli/tests/fn/sum.py', data=None, adapter='dml-python-in-process-adapter')"
    },
    {
      "id": "1a341262b3608b98f761512f466c8c00",
      "name": null,
      "doc": null,
      "type": "literal",
      "info": "1"
    },
    {
      "id": "05fc18f1b71569cb8952972bc86495b7",
      "name": "fn-resource",
      "doc": null,
      "type": "literal",
      "info": "Resource(uri='/Users/amn/c